In [1]:
import numpy as np
import pandas as pd
import patsy as ps

from statsmodels.sandbox.regression.gmm import IV2SLS
import os, sys
sys.path.append(os.path.abspath("../../../"))
from dowhy import CausalModel

In [2]:
n_points = 1000
education_abilty = 1
education_voucher = 0.5
income_abilty = 2
income_education = 4


# confounder
ability = np.random.normal(0, 3, size=n_points)

# instrument
voucher = np.random.normal(2, 1, size=n_points) 

# treatment
education = np.random.normal(5, 1, size=n_points) + education_abilty * ability +\
            education_voucher * voucher

# outcome
income = np.random.normal(10, 3, size=n_points) +\
         income_abilty * ability + income_education * education

# build dataset
data = np.stack([ability, education, income, voucher]).T
df = pd.DataFrame(data, columns = ['ability', 'education', 'income', 'voucher'])

In [3]:
income_vec, endog = ps.dmatrices("income ~ education", data=df)
exog = ps.dmatrix("voucher", data=df)

m = IV2SLS(income_vec, endog, exog).fit()
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                 income   R-squared:                       0.891
Model:                         IV2SLS   Adj. R-squared:                  0.891
Method:                     Two Stage   F-statistic:                     142.4
                        Least Squares   Prob (F-statistic):           8.70e-31
Date:                Fri, 08 Nov 2019                                         
Time:                        22:37:01                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.8927      2.132      4.171      0.000       4.709      13.076
education      4.2154      0.353     11.935      0.000       3.522       4.908
==============================================================================
Omnibus:                        0.127   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.938   Jarque-Bera (JB):                0.200
Skew:                           0.009   Prob(JB):                        0.905
Kurtosis:                       2.933   Cond. No.                         14.6
==============================================================================
"""

In [4]:
model=CausalModel(
        data = df,
        treatment='education',
        outcome='income',
        common_causes=['ability'],
        instruments=['voucher']
        )

identified_estimand = model.identify_effect()

estimate = model.estimate_effect(identified_estimand,
        method_name="iv.instrumental_variable", test_significance=True
)
print(estimate)


INFO:dowhy.causal_model:Model to find the causal effect of treatment ['education'] on outcome ['income']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['U', 'ability']


WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:['voucher']
INFO:dowhy.causal_estimator:INFO: Using Instrumental Variable Estimator
INFO:dowhy.causal_estimator:Realized estimand: Wald Estimator
Realized estimand type: ate
Estimand expression:
                                                                              
Expectation(Derivative(income, voucher))⋅Expectation(Derivative(education, vou

      -1
cher))  
Estimand assumption 1, treatment_effect_homogeneity: Each unit's treatment education isaffected in the same way by common causes of education and income
Estimand assumption 2, As-if-random: If U→→income then ¬(U →→voucher)
Estimand assumption 3, Exclusion: If we remove {voucher}→education, then ¬(voucher→income)
Estimand assumption 4, outcome_effect_homogeneity: Each unit's outcome income isaffected in the same way by common causes of education and income



*** Causal Estimate ***

## Target estimand
Estimand type: ate
### Estimand : 1
Estimand name: iv
Estimand expression:
Expectation(Derivative(income, voucher)/Derivative(education, voucher))
Estimand assumption 1, As-if-random: If U→→income then ¬(U →→voucher)
Estimand assumption 2, Exclusion: If we remove {voucher}→education, then ¬(voucher→income)
### Estimand : 2
Estimand name: backdoor
Estimand expression:
    d                                  
──────────(Expectation(income|ability))
deducation                             
Estimand assumption 1, Unconfoundedness: If U→education and U→income then P(income|education,ability,U) = P(income|education,ability)

## Realized estimand
Realized estimand: Wald Estimator
Realized estimand type: ate
Estimand expression:
                                                                              
Expectation(Derivative(income, voucher))⋅Expectation(Derivative(education, vou

      -1
cher))  
Estimand assumption 1, treatment_effect_homogeneit